In [8]:
import os, glob, h5py, math, time, json
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
from scipy.stats import norm, chi2

# ----- style -----
plt.rcParams["font.family"] = "serif"
plt.style.use('classic')
plt.rcParams['patch.edgecolor'] = 'none'
plt.rcParams['patch.linewidth'] = 0.0

# ----- chi2 Z-score like your reference -----
def Z_score_chi2(t, df):
    sf = chi2.sf(t, df)          # 1 - CDF
    Z  = -norm.ppf(sf)           # convert to one-sided Z
    return Z, sf                 # return (Z, p-value)

def plot_overlap_from_arrays(t1, t2, df, save_path, label1='REF', label2='DATA',
                             xmin=None, xmax=None, nbins=30, ymax=None, print_Z=True):
    t1 = np.asarray(t1, float);  t2 = np.asarray(t2, float)
    if xmin is None: xmin = float(np.min(t1))
    if xmax is None: xmax = float(np.max(t2))
    bins = np.linspace(xmin, xmax, nbins + 1)
    bw   = (xmax - xmin) / nbins
    xctr = 0.5 * (bins[1:] + bins[:-1])

    fig = plt.figure(figsize=(12, 9))
    fig.patch.set_facecolor('white')

    # ---- REF ----
    Zr, p_r = Z_score_chi2(np.median(t1), df)
    med_err_r = 1.2533 * np.std(t1) / np.sqrt(len(t1))
    Zr_p, _   = Z_score_chi2(np.median(t1) + med_err_r, df)
    Zr_m, _   = Z_score_chi2(np.median(t1) - med_err_r, df)

    lab_r = (f'{label1} \nsize: {len(t1)}\n'
             f'median: {np.median(t1):.2f}, std: {np.std(t1):.2f}\n'
             f'p-value: {p_r:.5f}\n')
    if print_Z:
        lab_r += f'asymptotic Z = {Zr:.2f} (+{Zr_p-Zr:.2f}/-{Zr-Zr_m:.2f})'

    h1 = plt.hist(t1, weights=np.ones_like(t1)/(len(t1)*bw),
                  bins=bins, color="#e186ed", alpha=0.5, label=lab_r,
                  edgecolor='none', linewidth=0)
    err1 = np.sqrt(h1[0] / (len(t1) * bw))
    plt.errorbar(xctr, h1[0], yerr=err1, color="#8a2be2",
                 marker='o', ls='', alpha=0.6, markersize=5, capsize=2, elinewidth=0.8)

    # ---- DATA ----
    Zd, p_d = Z_score_chi2(np.median(t2), df)
    med_err_d = 1.2533 * np.std(t2) / np.sqrt(len(t2))
    Zd_p, _   = Z_score_chi2(np.median(t2) + med_err_d, df)
    Zd_m, _   = Z_score_chi2(np.median(t2) - med_err_d, df)

    # empirical tail vs REF (optional but matches your earlier plot)
    t_emp = np.sum(t1 > np.mean(t2)) / len(t1)
    empirical_lim = '=' if t_emp > 0 else '>'
    if t_emp == 0: t_emp = 1.0 / len(t1)
    t_emp_err = t_emp * np.sqrt(1.0 / (np.sum(t1 > np.mean(t2)) + 1.0/len(t1)))
    Z_emp     = norm.ppf(1 - t_emp)
    Z_emp_m   = norm.ppf(1 - (t_emp + t_emp_err))
    Z_emp_p   = norm.ppf(1 - (t_emp - t_emp_err))

    lab_d = (f'{label2} \nsize: {len(t2)}\n'
             f'median: {np.median(t2):.2f}, std: {np.std(t2):.2f}\n'
             f'p-value: {p_d:.5f}\n')
    if print_Z:
        lab_d += (f'asymptotic Z = {Zd:.2f} (+{Zd_p-Zd:.2f}/-{Zd-Zd_m:.2f})\n'
                  f'empirical Z {empirical_lim} {Z_emp:.2f} '
                  f'(+{Z_emp_p-Z_emp:.2f}/-{Z_emp-Z_emp_m:.2f})')

    h2 = plt.hist(t2, weights=np.ones_like(t2)/(len(t2)*bw),
                  bins=bins, color="#68aedc", alpha=0.5, label=lab_d,
                  edgecolor='none', linewidth=0)
    err2 = np.sqrt(h2[0] / (len(t2) * bw))
    plt.errorbar(xctr, h2[0], yerr=err2, color="#004c99",
                 marker='o', ls='', alpha=0.6, markersize=5, capsize=2, elinewidth=0.8)

    # cosmetics
    font = font_manager.FontProperties(family='serif', size=16.5)
    plt.legend(ncol=1, loc='upper right', prop=font, frameon=False)
    plt.xlabel(r'$t$', fontsize=32, fontname="serif")
    plt.ylabel('Probability', fontsize=32, fontname="serif")
    if ymax is not None:
        plt.ylim(0., ymax)
    plt.yticks(fontsize=22, fontname="serif")
    plt.xticks(fontsize=22, fontname="serif")
    plt.tight_layout()

    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    plt.savefig(save_path, bbox_inches="tight", pad_inches=0.01)
    plt.close()

def plot_overlap_from_h5(ref_h5, data_h5, save_dir, nbins=30, label1='REF', label2='DATA'):
    # load arrays
    with h5py.File(ref_h5, 'r') as fr:
        key = ref_h5.split('flksigma')[-1].replace('.h5', '')
        t1  = np.array(fr[key])
    with h5py.File(data_h5, 'r') as fd:
        key = data_h5.split('flksigma')[-1].replace('.h5', '')
        t2  = np.array(fd[key])

    # df as mean of REF (your convention)
    df = float(np.mean(t1))

    # x-range like your loop
    xmin = float(np.min(t1))
    xmax = float(np.max(t2) + 50.0)

    # build filename tag
    params = '_'.join(os.path.basename(os.path.dirname(ref_h5)).split('_')[2:7])
    out = os.path.join(save_dir, f'probability_plot_overlap_{params}.pdf')

    plot_overlap_from_arrays(
        t1, t2, df=df, save_path=out, label1=label1, label2=label2,
        xmin=xmin, xmax=xmax, nbins=nbins, ymax=None, print_Z=True
    )
    return out

# -------- run on your inputs (one or all) --------
models = {
    2000: {
        'ref': '/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/NPLM/NPLM_NF_ensemble/calibration/nplm_ensemble_NR10000_NG2000_M500_lam1e-6_iter1000000_job233054/tvalues_flksigma1.0.h5',
        'data': '/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/NPLM/NPLM_NF_ensemble/comparison/nplm_ensemble_NR10000_NG2000_M500_lam1e-6_iter1000000_job233053/tvalues_flksigma1.0.h5'
    },
    5000: {
        'ref': '/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/NPLM/NPLM_NF_ensemble/calibration/nplm_ensemble_NR25000_NG5000_M700_lam1e-6_iter1000000_job233009/tvalues_flksigma1.0.h5',
        'data': '/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/NPLM/NPLM_NF_ensemble/comparison/nplm_ensemble_NR25000_NG5000_M700_lam1e-6_iter1000000_job233010/tvalues_flksigma1.0.h5'
    },
    10000: {
        'ref': '/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/NPLM/NPLM_NF_ensemble/calibration/nplm_ensemble_NR50000_NG10000_M1000_lam1e-6_iter1000000_job233008/tvalues_flksigma1.0.h5',
        'data': '/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/NPLM/NPLM_NF_ensemble/comparison/nplm_ensemble_NR50000_NG10000_M1000_lam1e-6_iter1000000_job233007/tvalues_flksigma1.0.h5'
    },
    15000: {
        'ref': '/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/NPLM/NPLM_NF_ensemble/calibration/nplm_ensemble_NR75000_NG15000_M1200_lam1e-6_iter1000000_job233005/tvalues_flksigma1.0.h5',
        'data': '/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/NPLM/NPLM_NF_ensemble/comparison/nplm_ensemble_NR75000_NG15000_M1200_lam1e-6_iter1000000_job233006/tvalues_flksigma1.0.h5'
    },
    20000: {
        'ref': '/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/NPLM/NPLM_NF_one_model/calibration/model_NR100000_NG20000_M1400_lam1e-6_iter1000000_job438523/tvalues_flksigma1.0.h5',
        'data': '/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/NPLM/NPLM_NF_one_model/comparison/model_NR100000_NG20000_M1400_lam1e-6_iter1000000_job428378/tvalues_flksigma1.0.h5'
    },
}

FIG_DIR = "./figs_overlap"
os.makedirs(FIG_DIR, exist_ok=True)

# choose ONE model (e.g., 10000) …
# out_file = plot_overlap_from_h5(models[10000]['ref'], models[10000]['data'], FIG_DIR, nbins=30)

# …or generate them ALL:
for key, p in models.items():
    out_file = plot_overlap_from_h5(p['ref'], p['data'], FIG_DIR, nbins=30)
    print(f"[{key}] saved: {out_file}")


[2000] saved: ./figs_overlap/probability_plot_overlap_NR10000_NG2000_M500_lam1e-6_iter1000000.pdf
[5000] saved: ./figs_overlap/probability_plot_overlap_NR25000_NG5000_M700_lam1e-6_iter1000000.pdf
[10000] saved: ./figs_overlap/probability_plot_overlap_NR50000_NG10000_M1000_lam1e-6_iter1000000.pdf
[15000] saved: ./figs_overlap/probability_plot_overlap_NR75000_NG15000_M1200_lam1e-6_iter1000000.pdf
[20000] saved: ./figs_overlap/probability_plot_overlap_NG20000_M1400_lam1e-6_iter1000000_job438523.pdf
